# Analytics

#### Date: 2021/01

#### SUMMARY:

- This notebook represents the project quality analysis of the date exposed right above. 

### TEAM:

##### Semester: 2021/01
##### Professor: Hilmer Neri

##### Members:

- Érico Maximiano Bandeira
- Henrique Martins de Messias
- João Vitor Moura Rosa
- Max Henrique Barbosa
- Victor Rodrigues Silva
- Antonio Igor Carvalho
- Gabriel Santos Silva Araújo
- João Paulo Lima da Silva
- Lucas Vieira de Jesus

### LIBRARIES

In [11]:
# Deal with data
import pandas as pd
import numpy as np
import json

# Deal with API request
import urllib3
from urllib3 import request

# Deal with visualization
import seaborn as sns
import matplotlib.pyplot as plt

### GRAPH SETTINGS

In [12]:
%config InlineBackend.figure_format ='retina'
sns.set(font_scale=1.5)
sns.set_style('darkgrid',
              {'xtick.bottom' : True,
               'ytick.left': True,
               'grid.linestyle':'--',
               'font.monospace': ['Computer Modern Typewriter'],
               'axes.edgecolor' : 'white'})

### DATAFRAME SETTINGS

In [13]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

### SonarCloud

#### KEYS

In [14]:
front_key = 'fga-eps-mds_2021-1-PC-GO-Frontend'

In [15]:
profile_key = 'fga-eps-mds_2021.1-Oraculo-Profile'

In [16]:
registros_key = 'fga-eps-mds_2021.1-Oraculo-Processos'

In [17]:
tags_key = 'fga-eps-mds_2021.1-Oraculo-Tags'

#### METRICS

In [18]:
metric_list = ['files',
               'functions',
               'complexity',
               'comment_lines_density',
               'duplicated_lines_density',
               'security_rating',
               'tests',
               'test_success_density',
               'test_execution_time',
               'reliability_rating']

len(metric_list)

10

In [19]:
def generate_metric_string(metric_list):
    metric_str = ''
    
    for metric in metric_list:
        metric_str += metric + ','
        
    return metric_str

In [20]:
metric_str = generate_metric_string(metric_list)

#### URLS

In [21]:
front_url = f"https://sonarcloud.io/api/measures/component_tree?component={front_key}&metricKeys={metric_str}&ps=500"
profile_url = f"https://sonarcloud.io/api/measures/component_tree?component={profile_key}&metricKeys={metric_str}&ps=500"
registros_url = f"https://sonarcloud.io/api/measures/component_tree?component={registros_key}&metricKeys={metric_str}&ps=500"
tags_url = f"https://sonarcloud.io/api/measures/component_tree?component={tags_key}&metricKeys={metric_str}&ps=500"

#### API REQUEST

In [22]:
http = urllib3.PoolManager()

In [23]:
front_request = http.request('GET', front_url)
front_request.status

200

In [24]:
profile_request = http.request('GET', profile_url)
profile_request.status

200

In [25]:
registros_request = http.request('GET', registros_url)
registros_request.status

200

In [26]:
tags_request = http.request('GET', tags_url)
tags_request.status

200

#### JSON DECODING

In [27]:
front_json = json.loads(front_request.data.decode('utf-8'))
profile_json = json.loads(profile_request.data.decode('utf-8'))
registros_json = json.loads(registros_request.data.decode('utf-8'))
tags_json = json.loads(tags_request.data.decode('utf-8'))

## DATA

### PROJECT

In [28]:
project_front_json = front_json['baseComponent']['measures']
project_profile_json = profile_json['baseComponent']['measures']
project_registros_json = registros_json['baseComponent']['measures']
project_tags_json = tags_json['baseComponent']['measures']

In [29]:
project_front_data = pd.DataFrame(project_front_json)
project_profile_data = pd.DataFrame(project_profile_json)
project_registros_data = pd.DataFrame(project_registros_json)
project_tags_data = pd.DataFrame(project_tags_json)

##### FRONT

In [30]:
project_front_data

,metric,value,bestValue
0,duplicated_lines_density,2.7,False
1,functions,82,NaN
2,security_rating,1.0,True
3,files,58,NaN
4,complexity,100,NaN
5,reliability_rating,1.0,True
6,comment_lines_density,1.4,False


##### BACK

In [31]:
project_profile_data

,metric,value,bestValue
0,duplicated_lines_density,0.0,True
1,functions,25,NaN
2,security_rating,1.0,True
3,files,12,NaN
4,complexity,44,NaN
5,reliability_rating,1.0,True
6,comment_lines_density,0.8,False


In [32]:
project_registros_data

,metric,value,bestValue
0,duplicated_lines_density,0.0,True
1,functions,26,NaN
2,security_rating,1.0,True
3,files,9,NaN
4,complexity,42,NaN
5,reliability_rating,1.0,True
6,comment_lines_density,4.1,False


In [33]:
project_tags_data

,metric,value,bestValue
0,duplicated_lines_density,0.0,True
1,functions,11,NaN
2,security_rating,1.0,True
3,files,7,NaN
4,complexity,13,NaN
5,reliability_rating,1.0,True
6,comment_lines_density,0.0,False


### FILES

In [34]:
def metric_per_file(json):
    file_json = []
    
    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)
            
    return file_json

In [35]:
front_file_json = metric_per_file(front_json)
profile_file_json = metric_per_file(profile_json)
registros_file_json = metric_per_file(registros_json)
tags_file_json = metric_per_file(tags_json)

In [36]:
def generate_file_dataframe(metric_list, json, language_extension):
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)
    
    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass
        
    df.reset_index(inplace = True)
    df = df.rename({'index': 'path'}, axis=1).drop(['files'], axis=1)

    return df

In [37]:
front_files_data = generate_file_dataframe(metric_list, front_file_json, language_extension = 'js')
profile_files_data = generate_file_dataframe(metric_list, profile_file_json, language_extension = 'js')
registros_files_data = generate_file_dataframe(metric_list, registros_file_json, language_extension = 'js')
tags_files_data = generate_file_dataframe(metric_list, tags_file_json, language_extension = 'js')

##### FRONT

In [38]:
front_files_data

,path,functions,complexity,comment_lines_density,duplicated_lines_density,security_rating,tests,test_success_density,test_execution_time,reliability_rating
0,src/App.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
1,src/App.test.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
2,src/Auth/Auth.js,3,3,0.0,0.0,1.0,NaN,100.0,NaN,1.0
3,src/Services/Axios/BaseService/baseService.js,6,13,0.0,0.0,1.0,NaN,100.0,NaN,1.0
4,src/Constants/baseUrls.js,0,0,66.7,0.0,1.0,NaN,100.0,NaN,1.0
5,src/Components/DropDownButton/DivSelectSetor.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
6,src/history.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
7,src/index.js,0,0,20.0,0.0,1.0,NaN,100.0,NaN,1.0
8,src/Pages/CreateProcess/index.js,14,14,0.0,0.0,1.0,NaN,100.0,NaN,1.0
9,src/Pages/LoginScreen/index.js,5,6,0.0,0.0,1.0,NaN,100.0,NaN,1.0


##### BACK

In [39]:
profile_files_data

,path,functions,complexity,comment_lines_density,duplicated_lines_density,security_rating,tests,test_success_density,test_execution_time,reliability_rating
0,src/Database/config/database.js,0,0,6.3,0.0,1.0,NaN,100.0,NaN,1.0
1,src/Model/Department.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
2,src/Utils/hash.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
3,src/index.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
4,src/Database/index.js,6,6,0.0,0.0,1.0,NaN,100.0,NaN,1.0
5,src/Utils/JWT.js,2,4,0.0,0.0,1.0,NaN,100.0,NaN,1.0
6,src/Model/Level.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
7,src/routes.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
8,src/Model/Section.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
9,src/Model/User.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0


In [40]:
registros_files_data

,path,functions,complexity,comment_lines_density,duplicated_lines_density,security_rating,tests,test_success_density,test_execution_time,reliability_rating
0,src/Model/Field.js,1,1,40.0,0.0,1.0,NaN,100.0,NaN,1.0
1,src/index.js,1,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
2,src/Database/index.js,7,13,2.6,0.0,1.0,NaN,100.0,NaN,1.0
3,src/Model/Record.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
4,src/Controller/RecordController.js,11,20,0.7,0.0,1.0,NaN,100.0,NaN,1.0
5,src/routes.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
6,src/Model/Section.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
7,src/Model/Situation.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0


In [41]:
tags_files_data

,path,functions,complexity,comment_lines_density,duplicated_lines_density,security_rating,tests,test_success_density,test_execution_time,reliability_rating
0,src/index.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
1,src/Database/index.js,6,6,0.0,0.0,1.0,NaN,100.0,NaN,1.0
2,src/Database/config/index.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
3,src/routes.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
4,src/Model/Tag.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
5,src/Controller/TagController.js,3,5,0.0,0.0,1.0,NaN,100.0,NaN,1.0


# ANALYSIS

## MAINTAINABILITY

### CODE QUALITY

##### COMPLEXITY

In [42]:
def m1(df):
    
    density_non_complex_files = round((len(df[(df['complexity'].astype(float)/df['functions'].astype(float)) < 10])/len(df))*100, 2)
    
    return density_non_complex_files

##### COMMENTS

In [43]:
def m2(df):
    
    density_comment_files = round((len(df[(df['comment_lines_density'].astype(float) > 10) & (df['comment_lines_density'].astype(float) < 30)])/len(df))*100, 2)
    
    return density_comment_files

##### DUPLICATIONS

In [44]:
def m3(df):
    
    duplication = round((len(df[(df['duplicated_lines_density'].astype(float) < 5)])/len(df))*100, 2)
    
    return duplication

### BLOCKING CODE

#### NON-BLOCKING FILES

In [45]:
def m4(df):
    
    non_blocking_files = round((len(df[(df['security_rating'].astype(float) >= 4)])/len(df))*100,2)
    
    return non_blocking_files

## RELIABILITY

#### TEST SUCCESS

In [46]:
def m5(df):
    
    test_success_file = df[['path', 'test_success_density']]
    test_success_repository = df['test_success_density'].astype(float).mean()
    
    print("Project test unit density: ", test_success_repository)
    
    return test_success_file

#### FAST TESTS

In [47]:
def m6(df):
    
    fast_test_df = df[(df['test_execution_time'].astype(float) < 300)]
    fast_test_df['fast_test'] = fast_test_df['test_execution_time']/fast_test_df['tests']
    
    fast_test_file = fast_test_df[['path', 'fast_test']]
    
    fast_test_repository = fast_test_df['fast_test'].astype(float).mean()
    
    print("Project test unit density: ", fast_test_repository)
    
    return fast_test_file

## PRODUCTIVITY

### TIMEFRAME: SPRINT 0

In [54]:
NUMBER_OF_ISSUES_RESOLVED = 43
NUMBER_OF_ISSUES = 54

TAGS = {
    "HOTFIX": 3,
    "DOCS": 16,
    "FEATURE": 6,
    "ARQ": 0,
    "DEVOPS": 20,
    "ANALYTICS": 5,
    "US": 7,
    "EASY": 7,
    "MEDIUM": 5,
    "HARD": 0,
    "EPS": 32,
    "MDS": 1
}

#### RESOLVED ISSUES' THROUGHPUT

In [73]:
def m7(number_of_issues_resolved, number_of_issues):
    
    resolved_issues_throughput = round((number_of_issues_resolved / number_of_issues) * 100, 2)
    
    return resolved_issues_throughput

#### ISSUE TYPE IN A TIMEFRAME

In [74]:
def density(issue, number_of_issues):
    issue_density = round((issue / number_of_issues) * 100, 2)
    return issue_density

In [75]:
def m8(tag_dict, number_of_issues):
    
    issue_densities = {
        "hotfix": [density(tag_dict["HOTFIX"], number_of_issues)],
        "docs": [density(tag_dict["DOCS"], number_of_issues)],
        "feature": [density(tag_dict["FEATURE"], number_of_issues)],
        "arq": [density(tag_dict["ARQ"], number_of_issues)],
        "devops": [density(tag_dict["DEVOPS"], number_of_issues)],
        "analytics": [density(tag_dict["ANALYTICS"], number_of_issues)],
        "us": [density(tag_dict["US"], number_of_issues)],
        "easy": [density(tag_dict["EASY"], number_of_issues)],
        "medium": [density(tag_dict["MEDIUM"], number_of_issues)],
        "hard": [density(tag_dict["HARD"], number_of_issues)],
        "eps": [density(tag_dict["EPS"], number_of_issues)],
        "mds": [density(tag_dict["MDS"], number_of_issues)]
    }

    issue_densities = pd.DataFrame.from_dict(issue_densities).T.reset_index()
    
    issue_densities.columns = ['density' ,'percentage']
    
    return issue_densities

#### BUGS RATIO

In [76]:
def m9(tag_dict, number_of_issues):
    
    bugs_ratio = round(((tag_dict["DOCS"] + tag_dict["FEATURE"] + tag_dict["ARQ"] + tag_dict["DEVOPS"] + tag_dict["ANALYTICS"]) / number_of_issues) * 100, 2)
    
    return bugs_ratio

### METRIC RESULTS

In [66]:
m1(back_files_data)

NameError: name 'back_files_data' is not defined

In [ ]:
m2(back_files_data)

9.68

In [ ]:
m3(back_files_data)

90.32

In [ ]:
m4(back_files_data)

0.0

In [ ]:
m5(back_files_data)

Project test unit density:  100.0


,path,test_success_density
0,packages/user/src/controller/AuthenticateUserS...,100.0
1,packages/user/src/controller/AuthResolve.js,100.0
2,packages/benefits/src/models/benefit.js,100.0
3,packages/benefits/src/controller/BenefitContro...,100.0
4,packages/benefits/src/test/BenefitsController....,100.0
5,packages/user/src/controller/ChangePasswordRes...,100.0
6,packages/user/src/controller/CheckCodeResolve.js,100.0
7,packages/news/src/test/ControllerTesting.spec.js,100.0
8,packages/user/src/test/ControllerTesting.spec.js,100.0
9,packages/benefits/src/controller/CreateBenefit...,100.0


In [ ]:
m6(back_files_data)

Project test unit density:  nan


,path,fast_test


In [77]:
m7(NUMBER_OF_ISSUES_RESOLVED, NUMBER_OF_ISSUES)

79.63

In [78]:
m8(TAGS, NUMBER_OF_ISSUES)

,density,percentage
0,hotfix,5.56
1,docs,29.63
2,feature,11.11
3,arq,0.00
4,devops,37.04
5,analytics,9.26
6,us,12.96
7,easy,12.96
8,medium,9.26
9,hard,0.00


In [79]:
m9(TAGS, NUMBER_OF_ISSUES)

87.04